# 3. 베이스라인 모델

- 앞에서 선별한 피처들을 제거해 베이스라인 모델을 만들자
- **LightBGM**: 마이스로소프트가 개발한 모델, 빠르면서 성능이 좋음

In [ ]:
import pandas as pd

# 데이터 경로
data_path = '/kaggle/input/porto-seguro-safe-driver-prediction/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

## 3-1. 피처 엔지니어링

### 3-1-1. 데이터 합치기

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1) # 타깃값 제거

In [ ]:
all_features = all_data.columns # 전체 피처
all_features

### 3-1-2. 명목형 피처 원-핫 인코딩

- 모든 명목형 피처에 원-핫 인코딩을 적용

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# 명목형 피처 추출
cat_features = [feature for feature in all_features if 'cat' in feature] 

onehot_encoder = OneHotEncoder() # 원-핫 인코더 객체 생성
# 인코딩
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features]) 

encoded_cat_matrix

- 원-핫 인코딩을 하니 열이 184개나 생김!

### 3-1-3. 필요 없는 피처 제거

In [ ]:
# 추가로 제거할 피처
drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin', 
                 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']

# '1) 명목형 피처, 2) calc 분류의 피처, 3) 추가 제거할 피처'를 제외한 피처
remaining_features = [feature for feature in all_features 
                      if ('cat' not in feature and 
                          'calc' not in feature and 
                          feature not in drop_features)]

In [ ]:
from scipy import sparse

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[remaining_features]),
                               encoded_cat_matrix],
                              format='csr')

### 3-1-4. 데이터 나누기

In [ ]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

## 3-2. 평가지표 계산 함수 작성

- 본 경진대회의 평가지표는 정규화된 지니계수
    - 파이썬이나 사이킷런에서 기본으로 제공하지 않으므로 직접 만들어야 함
    
### 3-2-1. 지니계수란?

- 경제학에서 지니계수는 소득 불평등 정도를 나타내는 지표
- 지니계수가 작을수록 소득 수준이 평등 / 클수록 불평등
- 로렌츠 곡선을 이용해 계산

### 3-2-2. 정규화 지니계수 계산 함수

- 정규화 지니계수 = 예측 값에 대한 지니 계수 / 예측이 완벽할 때의 지니 계수

In [ ]:
import numpy as np

def eval_gini(y_true, y_pred):
    # 실제값과 예측값의 크기가 같은지 확인 (값이 다르면 오류 발생)
    assert y_true.shape == y_pred.shape

    n_samples = y_true.shape[0]                      # 데이터 개수
    L_mid = np.linspace(1 / n_samples, 1, n_samples) # 대각선 값

    # 1) 예측값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()] # y_pred 크기순으로 y_true 값 정렬
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선
    G_pred = np.sum(L_mid - L_pred)       # 예측 값에 대한 지니계수

    # 2) 예측이 완벽할 때 지니계수
    true_order = y_true[y_true.argsort()] # y_true 크기순으로 y_true 값 정렬
    L_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true)       # 예측이 완벽할 때 지니계수

    # 정규화된 지니계수
    return G_pred / G_true

In [ ]:
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds), True # 반환값

## 3-3. 모델 훈련 및 성능 검증

### 3-3-1. OOF 예측 방식

- **OOF 예측 (Out of Fold prediction)**: K 폴드 교차 검증을 수행하면서 각 폴드마다 테스트 데이터로 예측하는 방식
    - K 폴드 교차 검증을 하면서 폴드마다
    1. 훈련 데이터로 모델을 훈련
    2. 검증 데이터로 모델 성능을 측정
    3. 테스트 데이터로 최종 타깃 확률도 예측
    
    
- **OOF 예측 절차**
    1. 전체 훈련 데이터를 K개 그룹으로 나눈다
    2. K개 그룹 중 한 그룹은 검증 데이터, 나머지 K-1개 그룹은 훈련 데이터로 지정
    3. 훈련 데이터로 모델을 훈련
    4. 훈련된 모델을 이용해 검증 데이터로 타깃 확률을 예측, 전체 테스트 데이터로도 타깃 확률을 예측
    5. 검증 데이터로 구한 예측 확률과 테스트 데이터로 구한 예측 확률을 기록
    6. 검증 데이터를 다른 그룹으로 바꿔가며 2~5번 전차를 총 K번 반복
    7. K개 그룹의 검증 데이터로 예측한 확률을 훈련 데이터 실제 타깃값과 비교해 성능 평가 점수를 계산. 
        - 이 점수로 모델 성능을 가늠
    8. 테스트 데이터로 구한 K개 예측 확률의 평균을 구한다. 이 값이 최종 예측 확률이며, 제출해야하는 값


- **OOF 예측 방식의 장점**
    1. 과대적합 방지 효과
    2. 앙상블 효과가 있어 모델 성능이 좋아짐
        - 단일 모델로 한 번만 예측하는 것이 아니라, K개 모델로 K번 예측해 평균을 하므로


### 3-3-2. OOF 방식으로 LightGBM 훈련

- 이제 베이스라인 모델, LightGBM,을 훈련하면서 OOF 예측도 실제로 해보자


- 먼저, 층화 K 폴드 교차 검증기를 생성
    - 타깃값이 불균형하므로 "K폴드" 보다는 "층화K폴드"가 좋음

In [ ]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

In [ ]:
params = {'objective': 'binary',
          'learning_rate': 0.01,
          'force_row_wise': True, #경고문을 없애려고 추가한 파라미터
          'random_state': 0}

In [ ]:
# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0]) 

In [ ]:
import lightgbm as lgb

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정 
    X_train, y_train = X[train_idx], y[train_idx] # 훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터

    # LightGBM 전용 데이터셋 생성 
    dtrain = lgb.Dataset(X_train, y_train) # LightGBM 전용 훈련 데이터셋
    dvalid = lgb.Dataset(X_valid, y_valid) # LightGBM 전용 검증 데이터셋

    # LightGBM 모델 훈련 
    lgb_model = lgb.train(params=params,        # 훈련용 하이퍼파라미터
                          train_set=dtrain,     # 훈련 데이터셋
                          num_boost_round=1000, # 부스팅 반복 횟수
                          valid_sets=dvalid,    # 성능 평가용 검증 데이터셋
                          feval=gini,           # 검증용 평가지표
                          early_stopping_rounds=100, # 조기종료 조건
                          verbose_eval=100)     # 100번째마다 점수 출력
    
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)
    
    # 검증 데이터 예측 확률에 대한 정규화 지니계수 
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수 : {gini_score}\n')

In [ ]:
submission['target'] = oof_test_preds
submission.to_csv('submission.csv')